# PhraseTagger
`PhraseTagger` can be used to tag sequencial attribute values of a layer. The result is an enveloping layer.
Let's create a Text object with `morph_analysis` layer.

In [1]:
from estnltk import Text

text = Text('Eestimaal tunnevad inimesed palju puudust päikesest ja energiast.').tag_layer(['morph_analysis'])
text.morph_analysis

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Eestimaal', [{'normalized_text': 'Eestimaal', 'lemma': 'Eestimaa', 'root': 'Eesti_maa', 'root_tokens': ['Eesti', 'maa'], 'ending': 'l', 'clitic': '', 'form': 'sg ad', 'partofspeech': 'H'}]),
Span('tunnevad', [{'normalized_text': 'tunnevad', 'lemma': 'tundma', 'root': 'tund', 'root_tokens': ['tund'], 'ending': 'vad', 'clitic': '', 'form': 'vad', 'partofspeech': 'V'}]),
Span('inimesed', [{'normalized_text': 'inimesed', 'lemma': 'inimene', 'root': 'inimene', 'root_tokens': ['inimene'], 'ending': 'd', 'clitic': '', 'form': 'pl n', 'partofspeech': 'S'}]),
Span('palju', [{'normalized_text': 'palju', 'lemma': 'palju', 'root': 'palju', 'root_tokens': ['palju'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('puudust', [{'normalized_text': 'puudust', 'lemma': 'puudus', 'root': 'puudus', 'root_tokens': ['puudus'], 'ending': 't', 'clitic': '', 'form': 'sg p', 'partofspeech': 'S'}]),
Span('päikesest', [{'normalized_text': 'päikesest', 'lemma': 'päike', 'root': 'päike', 'root_tokens': ['päike'], 'ending': 'st', 'clitic': '', 'form': 'sg el', 'partofspeech': 'S'}, {'normalized_text': 'päikesest', 'lemma': 'päikene', 'root': 'päikene', 'root_tokens': ['päikene'], 'ending': 'st', 'clitic': '', 'form': 'sg el', 'partofspeech': 'S'}]),
Span('ja', [{'normalized_text': 'ja', 'lemma': 'ja', 'root': 'ja', 'root_tokens': ['ja'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'J'}]),
Span('energiast', [{'normalized_text': 'energiast', 'lemma': 'energia', 'root': 'energia', 'root_tokens': ['energia'], 'ending': 'st', 'clitic': '', 'form': 'sg el', 'partofspeech': 'S'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

Use `PhraseTagger` to tag sequences of lemmas on that text. The lemma sequences are read from a file.

In [2]:
from estnltk.taggers import Vocabulary

vocabulary_file = 'phrase_vocabulary.csv'
Vocabulary.parse(vocabulary=vocabulary_file, key='_phrase_')

Vocabulary(key='_phrase_', len=3)

The decorator function must return `True` if the annotation is valid and `False` otherwise. If the decorator returns `False` then the annotation will not be added to the span. The span is added to the output layer only if it has at least one annotation.

In [3]:
from estnltk.taggers import PhraseTagger


def decorator(span, annotation):
    assert span is annotation.span
    annotation['attr_1'] = 'default_1'
    annotation['attr_2'] = len(span)
    annotation['_priority_'] = 1
    return True
    

tagger = PhraseTagger(output_layer='phrases',
                      input_layer='morph_analysis',
                      input_attribute='lemma',
                      vocabulary=vocabulary_file,
                      key='_phrase_',
                      output_attributes=['value', '_priority_', 'attr_1', 'attr_2', '_phrase_'],
                      decorator=decorator,
                      conflict_resolving_strategy='ALL',
                      priority_attribute='_priority_',
                      output_ambiguous=True)
tagger

PhraseTagger(input_layers=('morph_analysis',), output_layer=phrases, output_attributes=('value', '_priority_', 'attr_1', 'attr_2', '_phrase_'), input_attribute=lemma, vocabulary=Vocabulary(key='_phrase_', len=3), decorator=<function __main__.decorator>, conflict_resolving_strategy=ALL, priority_attribute=_priority_, output_ambiguous=True, ignore_case=False)

**output_layer** - name of the output layer<br>
**input_layer** - name of the input layer<br>
**input_attribute** - name of the input layer attribute<br>
**vocabulary** - input vocabulary: `str`, `list`, `dict` or `Vocabulary`<br>
**key** - vocabulary key<br>
**output_attributes** - list of output layer attributes<br>
**global_validator** - global validator function that takes two arguments (span and raw text) and returns bool<br>
**validator_attribute** - name of the in vocabulary that points to the validator function<br>
**decorator** - decorator function that takes two arguments (span and raw text) and returns a dict of attribute names and their values; overwrites the vocabulary<br>
**conflict_resolving_strategy** - conflict resolving strategy<br>
**priority_attribute** - name of the priority attribute in the vocabulary<br>
**output_ambiguous** - output ambiguous layer<br>

The vocabulary read from the csv file looks like this.

So single lemma 'päike' is tagged, sequencial lemmas 'tundma', 'inimene' are tagged twice since the vocabulary contains two interpretations for that phrase and also sequence of lemmas 'tundma', 'inimene', 'palju' is tagged:

In [4]:
tagger.tag(text)
text.phrases

Layer(name='phrases', attributes=('value', '_priority_', 'attr_1', 'attr_2', '_phrase_'), spans=SL[EnvelopingSpan(['tunnevad', 'inimesed'], [{'value': 'TI_1', '_priority_': 1, 'attr_1': 'default_1', 'attr_2': 2, '_phrase_': ('tundma', 'inimene')}, {'value': 'TI_2', '_priority_': 1, 'attr_1': 'default_1', 'attr_2': 2, '_phrase_': ('tundma', 'inimene')}]),
EnvelopingSpan(['tunnevad', 'inimesed', 'palju'], [{'value': 'TIP', '_priority_': 1, 'attr_1': 'default_1', 'attr_2': 3, '_phrase_': ('tundma', 'inimene', 'palju')}]),
EnvelopingSpan(['päikesest'], [{'value': 'P', '_priority_': 1, 'attr_1': 'default_1', 'attr_2': 1, '_phrase_': ('päike',)}])])

All of the spans are kept by conflict resolving because the priorities are all equal.

In the next two examples conflicts are resolved only by priority. The first example is case sensitive, the second is not.

In [5]:
text = Text('Suur ja väike.').tag_layer(['words'])
comma_tagger = PhraseTagger(input_layer='words',
                            output_layer='size',
                            output_ambiguous=True,
                            input_attribute='text',
                            output_attributes=[],
                            key='_phrase_',
                            priority_attribute='_priority_',
                            conflict_resolving_strategy='ALL',
                            vocabulary=[{'_priority_': 0, '_phrase_': ('SUUR', 'ja', 'väike')}, 
                                        {'_priority_': 1, '_phrase_': ('väike',)}],
                            ignore_case=False
                         )
comma_tagger.tag(text)
text.size

Layer(name='size', attributes=('_priority_',), spans=SL[EnvelopingSpan(['väike'], [{'_priority_': 1}])])

In [6]:
text = Text('Suur ja väike.').tag_layer(['words'])
comma_tagger = PhraseTagger(input_layer='words',
                            output_layer='size',
                            output_ambiguous=True,
                            input_attribute='text',
                            output_attributes=[],
                            key='_phrase_',
                            priority_attribute='_priority_',
                            conflict_resolving_strategy='ALL',
                            vocabulary=[{'_priority_': 0, '_phrase_': ('SUUR', 'ja', 'väike')}, 
                                        {'_priority_': 1, '_phrase_': ('väike',)}],
                            ignore_case=True
                         )
comma_tagger.tag(text)
text.size

Layer(name='size', attributes=('_priority_',), spans=SL[EnvelopingSpan(['Suur', 'ja', 'väike'], [{'_priority_': 0}])])

## One more example

In [7]:
from estnltk.taggers import PhraseTagger

phrase_list = [
               ('jalg',),
               ('vasak', 'jalg'),
               ('parem', 'jalg'),
               ('kops',),
               ('vasak', 'kops'),
               ('parem', 'kops'),
               ('kõõlus',),
               ('lihas',),
               ('maks',),
               ('neer',),
               ('parem', 'neer'),
               ('vasak', 'neer'),
               ('varvas',),
               ('suur', 'varvas'),
              ]
phrase_list = [{'match': p} for p in phrase_list]


latin_dict = {('suur', 'varvas'): 'hallux', ('kõõlus',):'tendo', ('kops',):'pulmo'}

def decorator(span, annotation):
    annotation['latin_term'] = latin_dict.get(annotation.match)
    return True


tagger = PhraseTagger(output_layer='body_parts',
                      input_layer='morph_analysis',
                      input_attribute='lemma',
                      vocabulary=phrase_list,
                      key='match',
                      output_attributes=('match', 'latin_term'),
                      decorator=decorator,
                      conflict_resolving_strategy= 'MAX',
                      ignore_case=True)
tagger

PhraseTagger(input_layers=('morph_analysis',), output_layer=body_parts, output_attributes=('match', 'latin_term'), input_attribute=lemma, vocabulary=Vocabulary(key='match', len=14), decorator=<function __main__.decorator>, conflict_resolving_strategy=MAX, priority_attribute=None, output_ambiguous=False, ignore_case=True)

In [8]:
from estnltk import Text

text_1 = Text('Patsient lasi jalga, sest vasaku jala suure varba pika \
               painutajalihase kõõluse rebend ajas tal kopsu üle maksa.')
text_1.analyse('morphology')

tagger.tag(text_1)
text_1['body_parts']

Layer(name='body_parts', attributes=('match', 'latin_term'), spans=SL[EnvelopingSpan(['vasaku', 'jala'], [{'match': ('vasak', 'jalg'), 'latin_term': None}]),
EnvelopingSpan(['suure', 'varba'], [{'match': ('suur', 'varvas'), 'latin_term': 'hallux'}]),
EnvelopingSpan(['kõõluse'], [{'match': ('kõõlus',), 'latin_term': 'tendo'}]),
EnvelopingSpan(['kopsu'], [{'match': ('kops',), 'latin_term': 'pulmo'}])])

Note that 'maksa' is not tagged. This is because Vabamorf thinks its lemma is 'maksma'.